In [1]:
import keras
from keras.layers import MaxPooling2D, Conv2D, Dense, Flatten, Dropout, Activation
from keras.models import Sequential
import cv2
import numpy as np
import os

Using TensorFlow backend.


In [2]:
INPUT_SHAPE = (128, 128)
POS_PATH = "new_pos"
NEG_PATH = "cropped_not_using_phone"

def img_process(image):
    img = cv2.imread(image)
    img = cv2.resize(img, INPUT_SHAPE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_n = np.expand_dims(img, axis = -1)
    return img_n

def load_dataset(path, size = 128):
    img_list = os.listdir(path)
    X = []
    for index, img in enumerate(img_list):
        img_l = os.path.join(path, img)
        image = cv2.imread(img_l)
        image = cv2.resize(image, (size, size))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X.append(image)
    return np.array(X, dtype = "float32")


In [3]:
X1 = load_dataset(POS_PATH)
print(X1.shape[0])
X2 = load_dataset(NEG_PATH)
print(X2.shape[0])
y1 = np.ones((X1.shape[0]))
y2 = np.zeros((X2.shape[0]))
X = np.vstack([X1, X2])
X /= 255
y = np.hstack([y1, y2])

97
92


In [4]:
INPUT_SHAPE = (128, 128, 3)
model = Sequential()
model.add(Conv2D(32, (3,3), padding = "same", input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.001)

opt = keras.optimizers.RMSprop(learning_rate=0.1)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 63, 63, 64)       

In [6]:
from sklearn.model_selection import train_test_split
idx = np.arange(y.shape[0])
np.random.seed(0)
np.random.shuffle(idx)
X = X[idx]
y = y[idx]
X.shape

(189, 128, 128, 3)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 26)


In [8]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.15,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")

In [9]:
H = model.fit_generator(
	aug.flow(X_train, y_train, batch_size=16),
	validation_data=(X_val, y_val),
	steps_per_epoch=X_train.shape[0] // 16,
	epochs= 50,
	verbose=1)

Epoch 1/50
10/10 [==============================] - 17s 2s/step - loss: 189220320.8027 - accuracy: 0.5130 - val_loss: 2055.7864 - val_accuracy: 0.5789
Epoch 2/50
10/10 [==============================] - 17s 2s/step - loss: 22835.9365 - accuracy: 0.4091 - val_loss: 4.4432 - val_accuracy: 0.4211
Epoch 3/50
10/10 [==============================] - 15s 2s/step - loss: 34.3962 - accuracy: 0.4805 - val_loss: 0.7366 - val_accuracy: 0.4211
Epoch 4/50
10/10 [==============================] - 17s 2s/step - loss: 0.6998 - accuracy: 0.4938 - val_loss: 0.6823 - val_accuracy: 0.5789
Epoch 5/50
10/10 [==============================] - 16s 2s/step - loss: 0.7487 - accuracy: 0.4416 - val_loss: 0.6814 - val_accuracy: 0.5789
Epoch 6/50
10/10 [==============================] - 16s 2s/step - loss: 2.6737 - accuracy: 0.4797 - val_loss: 0.6944 - val_accuracy: 0.4211
Epoch 7/50
10/10 [==============================] - 17s 2s/step - loss: 5.6098 - accuracy: 0.4875 - val_loss: 0.8894 - val_accuracy: 0.4211
Epoc